In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from rgb_matters.data.utils.label_loader import LabelLoader
from rgb_matters.data.utils.vis_label import vis_grasp
from rgb_matters.constant import GRASPNET_ROOT, LABEL_DIR

from graspnetAPI import GraspNet
import numpy as np
import matplotlib.pyplot as plt

# I renamed the folder from rgb_graspnet to rgb_matters to match the folder name
# I also installed with both pip and colon. Then it imported succesfully

In [3]:
camera = "realsense"
g = GraspNet(root=GRASPNET_ROOT, camera=camera, split="all")

Loading data path...: 100%|██████████| 190/190 [00:00<00:00, 712.67it/s]


In [ ]:
scene_id = 1
ann_id = 1

label_loader = LabelLoader(LABEL_DIR)
label = label_loader.load_label(scene_id=scene_id, camera=camera, ann_id=ann_id)

label_img = np.transpose(label, (1, 2, 0))

print(label.shape)
print(label_img.shape)


color = g.loadRGB(scene_id, camera, ann_id)
depth = g.loadDepth(scene_id, camera, ann_id)
mask = g.loadMask(scene_id, camera, ann_id)

print(color.shape)
print(depth.shape)
print(mask.shape)

# TODO figure out

(360, 72, 96)
(72, 96, 360)
(720, 1280, 3)
(720, 1280)
(720, 1280)


Right now we have 360 channels... 6 rotations by 60 views?



In [5]:
# Functinos to go between C and V-A index, taken from the library

from rgb_matters.data.utils.generate_anchor_matrix import NUM_VIEWS, NUM_ANGLES
def decode_view_angle_index(
    view_angle_index, num_views=NUM_VIEWS, num_angles=NUM_ANGLES
):
    """
    **Input:**

    - view_angle_index: int of the index of the view-angle combination.

    - num_views: int of the number of views.

    - num_angles: int of the number of angles.

    **Output:**

    - tuple of the index of view and angle
    """
    view_index = view_angle_index // num_angles
    angle_index = view_angle_index % num_angles
    return np.stack((view_index, angle_index)).T

def get_view_angle_index(
    view_index, angle_index, num_views=NUM_VIEWS, num_angles=NUM_ANGLES
):
    """
    **Input:**

    - view_index: int of the index of the view.

    - angle_index: int of the index of the angle.

    - num_views: int of the number of views.

    - num_angles: int of the number of angles.

    **Output:**

    - int of the index of the view-angle combination.
    """
    return view_index * num_angles + angle_index



view_angle_pairs = decode_view_angle_index(np.arange(NUM_VIEWS * NUM_ANGLES))
print(view_angle_pairs)

channel_index =get_view_angle_index(view_angle_pairs[:, 0], view_angle_pairs[:, 1])
print(channel_index)



[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 0  3]
 [ 0  4]
 [ 0  5]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 1  3]
 [ 1  4]
 [ 1  5]
 [ 2  0]
 [ 2  1]
 [ 2  2]
 [ 2  3]
 [ 2  4]
 [ 2  5]
 [ 3  0]
 [ 3  1]
 [ 3  2]
 [ 3  3]
 [ 3  4]
 [ 3  5]
 [ 4  0]
 [ 4  1]
 [ 4  2]
 [ 4  3]
 [ 4  4]
 [ 4  5]
 [ 5  0]
 [ 5  1]
 [ 5  2]
 [ 5  3]
 [ 5  4]
 [ 5  5]
 [ 6  0]
 [ 6  1]
 [ 6  2]
 [ 6  3]
 [ 6  4]
 [ 6  5]
 [ 7  0]
 [ 7  1]
 [ 7  2]
 [ 7  3]
 [ 7  4]
 [ 7  5]
 [ 8  0]
 [ 8  1]
 [ 8  2]
 [ 8  3]
 [ 8  4]
 [ 8  5]
 [ 9  0]
 [ 9  1]
 [ 9  2]
 [ 9  3]
 [ 9  4]
 [ 9  5]
 [10  0]
 [10  1]
 [10  2]
 [10  3]
 [10  4]
 [10  5]
 [11  0]
 [11  1]
 [11  2]
 [11  3]
 [11  4]
 [11  5]
 [12  0]
 [12  1]
 [12  2]
 [12  3]
 [12  4]
 [12  5]
 [13  0]
 [13  1]
 [13  2]
 [13  3]
 [13  4]
 [13  5]
 [14  0]
 [14  1]
 [14  2]
 [14  3]
 [14  4]
 [14  5]
 [15  0]
 [15  1]
 [15  2]
 [15  3]
 [15  4]
 [15  5]
 [16  0]
 [16  1]
 [16  2]
 [16  3]
 [16  4]
 [16  5]
 [17  0]
 [17  1]
 [17  2]
 [17  3]
 [17  4]
 [17  5]
 [18  0]
 [18  1]
 [18  2]
 

Ok here is the heart of the matter. The grasps are binned into based on their Rotation matrix and the anchor matrix.

So you need to then understand how the channel index is associated with the anchor matrix

``` python
anchor_matrix = generate_matrix()  # (120, 12, 3, 3)
# g = GraspNet(root = GRASPNET_ROOT, camera=camera, split='all')
grasp = graspnet.loadGrasp(
    sceneId=scene_id,
    annId=ann_id,
    format="6d",
    camera=camera,
    grasp_labels=grasp_labels,
    collision_labels=collision_labels,
    fric_coef_thresh=grasp_thresh,
)
################################
# import pickle
# f = open('grasp.pkl','rb')
# grasp = pickle.load(f)

# widths = np.array([],dtype=np.float32)
view_indexs = np.array([], dtype=np.float32)
angle_indexs = np.array([], dtype=np.float32)
gridxs = np.array([], dtype=np.float32)
gridys = np.array([], dtype=np.float32)
# offsetxs = np.array([],dtype=np.float32)
# offsetys = np.array([],dtype=np.float32)
# zs = np.array([],dtype=np.float32)
# for obj_id in tqdm(grasp.keys(),'generating grasp label'):
# for obj_id in grasp.keys():

# g_obj = grasp[obj_id]
# Rs = g_obj["Rs"]
Rs = grasp.rotation_matrices
# widths = np.hstack([widths,g_obj['widths']]) # width
points = grasp.translations
views_index, angles_index = get_towards_and_angles(Rs, anchor_matrix)
```


The anchor matrix defines..?

z axis means what? z with respect to what? hmmm...
The rotations are with respect to the table I belive? ok lets assume that for now? The camera moves... hmm perhaps its the z of the camera... 

Each channel is associate with a Rotation matrix? or an anchor matrix within the

In [6]:
# When just looking at the view and angles, it seemed that the first view was the most upright and slowly it went down to more in plane angles...
# I needed to however confirm what the form was in the actual anchor matrix that is used to compare agianst the gripper rotation matrix.

# What I intially found didn't make sense... once I started to plot the in axis rotations. What ever I pixed as the axis (I thought it was the z axis) was not the z axis, 
# as it shouldn't rotate around the z axis. So I ploted the full axis, and I saw it was the x axis which was being rotated around. So then if i switch the x and z axis of the rotation matrix
# it seemed to work out all right!

# This I guess makes sense beacuse the graspnet 1B label format as x along the approch direction
# Also in the paper they have AVH 1 as the most upright (from 1 to 360, so 1 = 0)
# Also Chat GPT things that view 0 - is the most top down.
# Also in the views, the most top down is view 0.

from rgb_matters.data.utils.generate_anchor_matrix import generate_matrix, generate_views, generate_angles

anchor_matrix = generate_matrix()
views = generate_views(NUM_VIEWS)
towards = -views
angles = generate_angles(NUM_ANGLES)
print(anchor_matrix.shape)
print(views.shape)
print(towards.shape)
print(np.round(towards, 2))
print(angles.shape)
print(np.rad2deg(angles))

(60, 6, 3, 3)
(60, 3)
(60, 3)
[[-0.13 -0.    0.99]
 [ 0.16  0.15  0.98]
 [-0.02 -0.28  0.96]
 [-0.2   0.27  0.94]
 [ 0.37 -0.07  0.92]
 [-0.35 -0.22  0.91]
 [ 0.12  0.44  0.89]
 [ 0.22 -0.43  0.88]
 [-0.48  0.18  0.86]
 [ 0.5   0.21  0.84]
 [-0.24 -0.51  0.82]
 [-0.18  0.56  0.81]
 [ 0.53 -0.31  0.79]
 [-0.62 -0.14  0.78]
 [ 0.37  0.53  0.76]
 [ 0.09 -0.67  0.74]
 [-0.53  0.44  0.72]
 [ 0.71  0.03  0.71]
 [-0.51 -0.51  0.69]
 [ 0.03  0.74  0.68]
 [ 0.48 -0.58  0.66]
 [-0.76  0.1   0.64]
 [ 0.64  0.45  0.62]
 [-0.17 -0.77  0.61]
 [-0.4   0.7   0.59]
 [ 0.78 -0.25  0.58]
 [-0.75 -0.35  0.56]
 [ 0.32  0.78  0.54]
 [ 0.29 -0.8   0.52]
 [-0.76  0.4   0.51]
 [ 0.84  0.22  0.49]
 [-0.48 -0.74  0.48]
 [-0.15  0.88  0.46]
 [ 0.71 -0.55  0.44]
 [-0.9  -0.07  0.42]
 [ 0.62  0.67  0.41]
 [-0.   -0.92  0.39]
 [-0.62  0.69  0.38]
 [ 0.93 -0.09  0.36]
 [-0.75 -0.57  0.34]
 [ 0.17  0.93  0.32]
 [ 0.51 -0.81  0.31]
 [-0.92  0.25  0.29]
 [ 0.86  0.44  0.27]
 [-0.34 -0.91  0.26]
 [-0.37  0.9   0.24]
 [ 0

In [8]:
# Just read off the z value above ^
# Or here we compare the distance to the upwards vector and see the same results
# Careful, while it seems they generate the views with the z axis, the actual anchor matrix seems to have x along the z axis. as per the Graspnet 1B grasp coord frame
target_towards = np.array([0, 0, 1])
views = generate_views(NUM_VIEWS)
towards = -views
diff = towards - target_towards
diff_norm = np.linalg.norm(diff, axis=1)
print(diff_norm)


[0.1290994  0.22360665 0.28867522 0.34156506 0.3872983  0.42817451
 0.46547472 0.5        0.53229063 0.5627314  0.59160798 0.6191391
 0.64549718 0.67082038 0.6952218  0.71879524 0.74161981 0.76376255
 0.78528131 0.80622574 0.82663985 0.84656169 0.86602538 0.88506115
 0.90369609 0.92195439 0.93985814 0.9574271  0.97467946 0.99163167
 1.00829893 1.02469502 1.04083305 1.05672452 1.07238051 1.08781122
 1.10302615 1.11803397 1.13284301 1.14746097 1.16189503 1.17615195
 1.19023802 1.20415945 1.21792175 1.23153022 1.24498993 1.25830575
 1.27148208 1.28452323 1.29743332 1.31021627 1.32287569 1.33541506
 1.34783778 1.36014705 1.37234592 1.38443732 1.39642401 1.40830869]


Ok the interpertation is actually very nice... each view move continuall towards flat. the first view is the most upright. its kinda like a spirally shape.

To verify this I can plot the z axis of each rotation matrix, in a color going from red to blue.

In [10]:
# A first attempt and visualizing the anchor matrix
# I select the z axis and plot lines

# Problems...
# Lines are very small hard to see: solution, lets plot full axis arrows
# Also, I didn't realise this yet, but the z_axis is not actually the z axis, its the x axis.

import open3d as o3d

anchor_matrix = generate_matrix()
print(anchor_matrix.shape)

# Get the z axis of the first angle for each view
z_axis = anchor_matrix[:, 0, :, 2]  # shape: (num_views, 3)
print(z_axis.shape)
print(z_axis)

# Visualize each z axis as a line from origin to z_axis, colored from red to blue
num_views = z_axis.shape[0]
lines = []
colors = []
points = []

for i in range(num_views):
    start = np.array([0, 0, 0])
    end = z_axis[i]
    points.append(start)
    points.append(end)
    lines.append([2 * i, 2 * i + 1])
    # Color interpolation: red (1,0,0) to blue (0,0,1)
    t = i / (num_views - 1)
    color = [1 - t, 0, t]
    colors.append(color)

line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)

# Draw the origin frame
origin_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])

o3d.visualization.draw_geometries([line_set, origin_frame])



(60, 6, 3, 3)
(60, 3)
[[ 0.9916667   0.          0.12883016]
 [-0.7189345  -0.6586032   0.22220472]
 [ 0.08378249  0.95466393  0.28565234]
 [ 0.57294697 -0.74730706  0.33654705]
 [-0.9108601   0.16111733  0.37996703]
 [ 0.7664107   0.4875284   0.41824707]
 [-0.23147954 -0.86109614  0.45269257]
 [-0.4032949   0.7765167   0.4841229 ]
 [ 0.8062514  -0.2944398   0.51309246]
 [-0.77799004 -0.32114533  0.5399974 ]
 [ 0.34967205  0.74723125  0.56513274]
 [ 0.24192396 -0.7712819   0.5887251 ]
 [-0.6849598   0.39694616  0.6109533 ]
 [ 0.75692296  0.16641106  0.6319612 ]
 [-0.4361378  -0.62036544  0.651867  ]
 [-0.09531346  0.7355168   0.6707687 ]
 [ 0.5543725  -0.46722174  0.68874884]
 [-0.70772845 -0.02926879  0.7058781 ]
 [ 0.49027115  0.48789036  0.72221696]
 [-0.03117922 -0.6742795   0.73781765]
 [-0.42180136  0.5054565   0.75272655]
 [ 0.6359366  -0.08556124  0.7669836 ]
 [-0.51303387 -0.35696113  0.78062475]
 [ 0.13351665  0.5935005   0.7936817 ]
 [ 0.29416764 -0.5133565   0.8061827 ]
 [-

In [12]:
# Now we plot just the first angle of the anchor matrix, using larger arrows, looks great!

DISPLAY_N_VIEWS = 60

anchor_matrix = generate_matrix()
n_views, n_angles, _, _ = anchor_matrix.shape
print(anchor_matrix.shape)

# Get the z axis of the first angle for each view
z_axis = anchor_matrix[:, 0, :, 2]  # shape: (num_views, 3)
print(z_axis.shape)


# Visualize each z axis as a line from origin to z_axis, colored from red to blue
display_n_views = min(n_views, DISPLAY_N_VIEWS)

geometries = []

for i in range(display_n_views):

    # if i not in [59]:
    #     continue

    R = anchor_matrix[i, 0, :, :]
    scale = 0.1
    arrow = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.005,
        cone_radius=0.01,
        cylinder_height=scale * 0.8,
        cone_height=scale * 0.2,
        resolution=20,
        cylinder_split=4,
        cone_split=1
    )
    t = i / (display_n_views - 1)
    color = [1 - t, 0, t]
    # Rotate the arrow directly using the full rotation matrix
    arrow.rotate(R, center=(0, 0, 0))
    arrow.translate(R[:, 2] * scale * 0.1)  # small offset along Z
    arrow.paint_uniform_color(color)
    geometries.append(arrow)


# Draw the origin frame
origin_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])

o3d.visualization.draw_geometries([*geometries, origin_frame])






(60, 6, 3, 3)
(60, 3)


In [21]:
# Now we plot the view and axises!
# If you comment out the rotation below you will see its wrong though, to coord system are different
DISPLAY_N_VIEWS = 60

anchor_matrix = generate_matrix()
n_views, n_angles, _, _ = anchor_matrix.shape

CHANNELS = NUM_VIEWS * NUM_ANGLES

view_angle_pairs = decode_view_angle_index(np.arange(CHANNELS))

geometries = []

for i, (view, angle) in enumerate(view_angle_pairs):
  
    # Its helpful to change this to select just the channels you want to select. 
    # You can read the values below...
    # if view not in [0, 1, 2, 3, 4, 5, 6]:
    #     continue
    if view not in [0, 1, 2]:
        continue
    print(f"i: {i}, view: {view}, angle: {angle}")
    R = anchor_matrix[view, angle, :, :]
    
    scale = 0.1
    arrow = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.005,
        cone_radius=0.01,
        cylinder_height=scale * 0.8,
        cone_height=scale * 0.2,
        resolution=20,
        cylinder_split=4,
        cone_split=1
    )
    t = i / (CHANNELS - 1)
    color = [1 - t, 0, t]

    # Rotate the arrow directly using the full rotation matrix
    # Switch the x and z axes of the rotation matrix R
    # This is done by pre- and post-multiplying R with a permutation matrix P
    # that swaps x and z axes: P = [[0,0,1],[0,1,0],[1,0,0]]
    P = np.array([[0, 0, 1],
                  [0, 1, 0],
                  [1, 0, 0]])
    R = P @ R @ P

    arrow.rotate(R, center=(0, 0, 0))
    arrow.translate(R[:, 2] * scale * 0.1)  # small offset along Z
    arrow.paint_uniform_color(color)
    geometries.append(arrow)


# Draw the origin frame
origin_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])

o3d.visualization.draw_geometries([*geometries, origin_frame])






i: 0, view: 0, angle: 0
i: 1, view: 0, angle: 1
i: 2, view: 0, angle: 2
i: 3, view: 0, angle: 3
i: 4, view: 0, angle: 4
i: 5, view: 0, angle: 5
i: 6, view: 1, angle: 0
i: 7, view: 1, angle: 1
i: 8, view: 1, angle: 2
i: 9, view: 1, angle: 3
i: 10, view: 1, angle: 4
i: 11, view: 1, angle: 5
i: 12, view: 2, angle: 0
i: 13, view: 2, angle: 1
i: 14, view: 2, angle: 2
i: 15, view: 2, angle: 3
i: 16, view: 2, angle: 4
i: 17, view: 2, angle: 5


In [ ]:
# Very Important, this helps you understand the coordinate system of the anchor matrix.
# Plot all 3 axises and see which one doesn't change as you change the angle view.
anchor_matrix = generate_matrix()
n_views, n_angles, _, _ = anchor_matrix.shape

CHANNELS = n_views * n_angles

view_angle_pairs = decode_view_angle_index(np.arange(CHANNELS))

origin_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])

for view in range(n_views):
    geometries = []

    for angle in range(n_angles):
        print(view, angle)
        R = anchor_matrix[view, angle, :, :]

        frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
        frame.rotate(R, center=(0, 0, 0))
        geometries.append(frame)


    o3d.visualization.draw_geometries([*geometries, origin_frame])
    break


0 0
0 1
0 2
0 3
0 4
0 5
